In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer,BertModel
import torch
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import re

In [18]:
tqdm.pandas()

In [3]:
# data_path="/kaggle/input/bus-emotion-gz/20188_new.csv"
data_path="/kaggle/input/bus-emo-900/bus_emo_900.csv"

In [4]:
# texts,labels=[],[]
# with open(data_path,'r',encoding='gbk',errors='ignore') as f:
#     while True:
#         line=f.readline()
#         if line:
#             line=line.strip().split(',')
#             if len(line[-2])==0:
#                 break
#             texts.append(line[-2])
#             labels.append(line[-1])
#         else:
#             break
# dic={}
# dic['text'],dic['label']=texts,labels
# df=pd.DataFrame(dic)

# #数据去重
# df=df.drop_duplicates()

# train_df,valid_df=train_test_split(df,test_size=0.2,random_state=1)

# #需要对数据的长度进行处理，长度太过参差
# #工业上一般短文本和长文本使用不同的判断手段
# df['text'].apply(len).describe()

In [34]:
# df.to_csv("./bus_emo.csv",index=False)

In [9]:
# df.groupby('label').count()

,text
label,
0,5188
1,3168


In [4]:
df=pd.read_csv(data_path,encoding='gbk')
#数据去重
df=df.drop_duplicates()
train_df,valid_df=train_test_split(df,test_size=0.2,random_state=1)

数据清理：

- 水话清除
  - #A#
  - @

- 括号转换：【A】->A，...

In [6]:
df

,text,label
0,广州公交，给力！,1
1,社会主义式垃圾袋——蚊帐广州有些新公交座椅旁边有隔板可以放东西，贴心,1
2,//@NLTV爆炸头小子:喜大普奔，中山市三角镇至广州南沙自贸区公交专线今日正式开通。,1
3,喜大普奔，中山三角镇至广州南沙自贸区公交专线今日正式开通！@葛雨晴-miyoyo@鞠婧祎,1
4,"司机师傅一边开着新公交车一边满脸自豪地跟乘客们介绍站点和地铁口?""",1
...,...,...
894,看到广州现在上下班高峰的人流，无论是在挤地铁还是在挤公交，真的毫无幸福感可言，那个人流密集度...,0
895,每天在奥林匹克中心等325和高峰63的人那么多，为什么就不能多安排几辆车呢？@广州公交集团每...,0
896,plk233近日，广州78路公交，司机关闭车门启动车辆后，一名女乘客发现自己坐过站，要求马上...,0
897,我第一次用公交车乘车码是在广州，不过早期地铁比较麻烦，就没有用，有一点体验不好，无法接受，马...,1


In [5]:
def char_change(text):
    if "【" in text:
        lst=list(text)
        s=lst.index('【')
        e=0
        if '】' in text:
            e=lst.index('】')
        if e==0:
            lst.remove('【')
        else:
            if s==0:
                lst[e]='。'
                lst.remove('【')
            else:
                lst[s]='。'
                lst[e]='。'
        return ''.join(lst)
    elif "】" in text:
        lst=list(text)
        lst.remove('】')
        return ''.join(lst)
    return text

In [6]:
def clean_suffix(text):
    #主要用来清除@和#后面的文字
    chars=":：!！?？,，.。@#（）()<>《》“”"";；"
    lst=list(text)
    while '@' in lst:
        idx=lst.index('@')
        i=idx+1
        e=-1
        while i<len(lst):
            if lst[i] in chars:
                e=i-1
                break
            if i==len(lst)-1:
                e=i
            i+=1
        if e>=0:
            tmp=lst[:idx][:]+lst[e+1:][:]
            lst=tmp[:]
        else:
            break
    if len(lst)==0:
        return text
    text=''.join(lst)
    if '#' in text:
        pattern=re.compile('\#.*\#')
        text=re.sub(pattern,'',text)
    return text

In [7]:
def clean_url(text):
    sentences = text.split(' ')
    # 处理http://类链接
    url_pattern = re.compile(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%|\-)*\b', re.S)
    # 处理无http://类链接
    domain_pattern = re.compile(r'(\b)*(.*?)\.(com|cn)')
    if len(sentences) > 0:
        result = []
        for item in sentences:
            text = re.sub(url_pattern, '', item)
            text = re.sub(domain_pattern,'', text)
            result.append(text)
        return ' '.join(result)
    else:
        return re.sub(url_pattern, '', sentences)

In [8]:
def clean_html(text):
    html_pattern = re.compile('</?\w+[^>]*>', re.S)
    text=re.sub(html_pattern,'', text)
    return text

In [9]:
def clean_tag(text):
    tag_pattern = re.compile('(\[|\#|【)(.*?)(\#|\]|\】)', re.S)
    text = re.sub(tag_pattern, '', text)
    return text.strip()

In [10]:
def clean_at(text):
    #暂时不用，因为有的@出现在句中或靠前，此时就会将后面的所有字都去除
    at_pattern = re.compile('@\S*', re.S)
    text = re.sub(at_pattern, '', text)
    return text.strip()

In [11]:
def clean_nan(text,label):
    new_text,new_label=[],[]
    for i in range(len(label)):
        text[i]=text[i].strip()
        if text[i]!='':
            new_text.append(text[i])
            new_label.append(label[i])
    return new_text,new_label

In [12]:
texts_df=train_df['text'].progress_apply(char_change)
texts_df=texts_df.progress_apply(clean_suffix)
texts_df=texts_df.progress_apply(clean_url)
texts_df=texts_df.progress_apply(clean_html)
labels_df=train_df['label']

100%|██████████| 719/719 [00:00<00:00, 139933.39it/s]


In [13]:
train_texts,train_labels=clean_nan(texts_df.values,labels_df.values)

In [14]:
test_texts=valid_df['text'].apply(char_change)
test_texts=test_texts.progress_apply(clean_suffix)
test_texts=test_texts.progress_apply(clean_url)
test_texts=test_texts.progress_apply(clean_html)
test_labels=valid_df['label']

100%|██████████| 180/180 [00:00<00:00, 91779.08it/s]


In [15]:
test_texts,test_labels=clean_nan(test_texts.values,test_labels.values)

In [16]:
tokenizer=BertTokenizer.from_pretrained("bert-base-chinese")

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [17]:
class Bert(nn.Module):
    def __init__(self,num_class,emb_size,max_length):
        super(Bert,self).__init__()
        self.max_length=max_length
        self.bert=BertModel.from_pretrained("bert-base-chinese")
        self.dropout=nn.Dropout(0.3)
        self.linear=nn.Linear(emb_size,num_class)
    
    def forward(self,inputs):
        data_token=tokenizer.batch_encode_plus(inputs,
                                               padding=True,
                                               truncation=True,
                                               max_length=self.max_length)
        
        input_ids=torch.tensor(data_token["input_ids"]).to(device)
        attention_mask=torch.tensor(data_token["attention_mask"]).to(device)
        token_type_ids=torch.tensor(data_token["token_type_ids"]).to(device)
        encode=self.bert(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        output=self.dropout(encode[0][:,0,:])
        output=self.linear(output)
        return output

In [57]:
class BertLstm(nn.Module):
    def __init__(self,num_class,emb_size,max_length,unit):
        super(BertLstm,self).__init__()
        self.max_length=max_length
        self.bert=BertModel.from_pretrained("bert-base-chinese")
        self.bilstm=nn.LSTM(input_size=emb_size,hidden_size=unit,bidirectional=True)
        self.dropout=nn.Dropout(0.3)
        self.linear=nn.Linear(unit*2,num_class)
    
    def forward(self,inputs):
        data_token=tokenizer.batch_encode_plus(inputs,
                                               padding=True,
                                               truncation=True,
                                               max_length=self.max_length)
        
        input_ids=torch.tensor(data_token["input_ids"]).to(device)
        attention_mask=torch.tensor(data_token["attention_mask"]).to(device)
        token_type_ids=torch.tensor(data_token["token_type_ids"]).to(device)
        encode=self.bert(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        output=self.bilstm(encode[0][:,0,:])
        output=self.dropout(output[0])
        output=self.linear(output)
        return output

In [69]:
class TextCNN(nn.Module):
    def __init__(self,filter_sizes,emb_size,in_channels,num_filter):
        super(TextCNN,self).__init__()
        self.convlist=nn.ModuleList([nn.Conv2d(in_channels=in_channels,
                                 out_channels=num_filter,
                                 kernel_size=[filter_size,emb_size]) for filter_size in filter_sizes])
        
    def forward(self,inputs):
        encodes=[]
        inputs=inputs.unsqueeze(1)
        for conv in self.convlist:
            output=conv(inputs)
            output=F.relu(output)
            output=output.squeeze(3)
            output=F.max_pool1d(output, output.shape[2]).squeeze(2)
            encodes.append(output)
        encode=torch.concat(encodes,dim=-1)
        return encode

In [70]:
class bert_textcnn(nn.Module):
    def __init__(self,num_filter,filter_sizes,in_channels,num_class,emb_size,max_length):
        super(bert_textcnn,self).__init__()
        self.max_length=max_length
        self.bert=BertModel.from_pretrained("bert-base-chinese")
        self.textcnn=TextCNN(filter_sizes=filter_sizes,
                             emb_size=emb_size,
                             in_channels=in_channels,
                             num_filter=num_filter)
        self.textcnn.to(device)
        self.dropout=nn.Dropout(0.3)
        self.linear=nn.Linear(num_filter*len(filter_sizes),num_class)
    
    def forward(self,inputs):
        token_dic=tokenizer.batch_encode_plus(inputs,
                                                padding=True,
                                                truncation=True,
                                                max_length=self.max_length)
        input_ids=torch.tensor(token_dic["input_ids"]).to(device)
        attention_mask=torch.tensor(token_dic["attention_mask"]).to(device)
        token_type_ids=torch.tensor(token_dic["token_type_ids"]).to(device)
        bert_output=self.bert(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids)
        last_hidden_out=bert_output[0][:,:,:]
        textcnn_output=self.textcnn(last_hidden_out)
        output=self.dropout(textcnn_output)
        output=self.linear(output)
        return output

In [42]:
#bert-base
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert=Bert(2,768,128)
bert.to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Bert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [60]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_bilstm=BertLstm(num_class=2,emb_size=768,max_length=150,unit=128)
bert_bilstm.to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertLstm(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [71]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_textcnn=bert_textcnn(num_filter=128,
                          filter_sizes=[3,4,5],
                          in_channels=1,
                          num_class=2,
                          emb_size=768,
                          max_length=150)
bert_textcnn.to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert_textcnn(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [19]:
print(next(textcnn.parameters()).device)

cuda:0


In [101]:
bert_textcnn(['#2019春运广州#//@广州交通:#温馨提示#目前，广州公交集团已在昌岗、公园前已安排应急出租车各30台。深夜乘坐地铁二号线夜间加班线路的旅客，可以在此两站出站换乘出租车。祝您出行顺利！@广州交通电台@FM1052羊城交通台'])

tensor([[-0.3413, -0.0596]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [42]:
class mydataset(torch.utils.data.Dataset):
    def __init__(self,data):
        self.texts=data[0]
        self.labels=list(map(int,data[1]))
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,idx):
        return self.texts[idx],self.labels[idx]

In [72]:
dataset=mydataset((train_texts,train_labels))
dataloader=torch.utils.data.DataLoader(dataset,batch_size=2,shuffle=True)

In [54]:
#分层学习率
optim_group=[
    {"params":[p for n,p in bert_textcnn.named_parameters() if 'bert' in n],"lr":1e-5},
    {"params":[p for n,p in bert_textcnn.named_parameters() if 'bert' not in n],"lr":1e-4}
]

In [73]:
#统一学习率
lf=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(bert_textcnn.parameters(),lr=1e-5,weight_decay=0.01)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer=optimizer,step_size=len(dataloader),gamma=0.6)

In [55]:
#分层学习率
lf=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(optim_group,weight_decay=0.01)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer=optimizer,step_size=len(dataloader),gamma=0.6)

In [68]:
def notrain(model,test_data,num_pos,num_neg):
    texts,labels=test_data[0],test_data[1]
    dataloader=torch.utils.data.DataLoader(mydataset((texts,labels)),batch_size=1,shuffle=False)
    pos,neg=0,0
    for i,batch in enumerate(dataloader):
        text,label=batch[0],batch[1]
        probas=model(text)
        pred=torch.argmax(probas,dim=1)
        label=label.numpy()[0]
        pred=pred.detach().cpu().numpy()[0]
        if pred==label:
            if pred==1:
                pos+=1
            if pred==0:
                neg+=1
        print("\rprocess:{}/{}".format(i+1,len(dataloader)),end='')
    print()
    print("正类的准确率：",pos/num_pos)
    print("负类的准确率：",neg/num_neg)
    print("整体的准确率：",(pos+neg)/len(dataloader))

In [64]:
def train(model):
#     model.train()
    for epoch in range(5):
        model.train()#因为每个epoch都会进行验证了，此时模型进入了eval模式，所有需要将train模型放到循环内部
        print('='*10,"epoch:{}/{}".format(epoch+1,5),'='*10)
        total_loss=0
        for i,batch in enumerate(dataloader):
            optimizer.zero_grad()
            text,label=batch[0],batch[1]
            label=label.to(device)
            output=model(text)
            loss=lf(output,label).to(device)
            total_loss+=loss.item()
            loss.backward()
            optimizer.step()
            if i%10==0:
                print("\rprocess:{}/{} | loss={} | lr={}".format(i+1,len(dataloader),total_loss/10,scheduler.get_lr()[0]),end='')
                total_loss=0
            scheduler.step()
        print()
        #根据loss保存最优模型，每个epoch结束后都进行一次预测
        print("开始预测...")
        predict(epoch+1,model)
    return model

In [65]:
def test(model,test_data,num_neg,num_pos,epoch):
    bad_text,bad_pred,bad_label=[],[],[]
    texts,labels=test_data[0],test_data[1]
    dataloader=torch.utils.data.DataLoader(mydataset((texts,labels)),batch_size=1,shuffle=False)
    model.eval()
    pos,neg=0,0
    with torch.no_grad():
        for i,batch in enumerate(dataloader):
            text,label=batch[0],batch[1]
            probas=model(text)
            pred=torch.argmax(probas,dim=1)
            label=label.numpy()[0]
            pred=pred.detach().cpu().numpy()[0]
            if pred==label:
                if pred==0:
                    neg+=1
                if pred==1:
                    pos+=1
            else:
                #输出bad case
                bad_text.append(text[0])
                bad_pred.append(pred)
                bad_label.append(label)
            print("\rprocess:{}/{}".format(i+1,len(dataloader)),end='')
    print()
    #保存bad case文件
    bad_df=pd.DataFrame({'text':bad_text,'label':bad_label,'pred':bad_pred})
    bad_df.to_csv('./bad_case_{}_5e-6.csv'.format(epoch),index=False)
    print("正类的准确率：",pos/num_pos)
    print("负类的准确率：",neg/num_neg)
    print("整体的准确率：",(pos+neg)/len(dataloader))

In [35]:
# def predict(epoch):
#     num_pos,num_neg=0,0
#     for i in test_labels:
#         print(i,type(i))
#         if i=='0':
#             num_neg+=1
#         elif i=='1':
#             num_pos+=1
#         else:
#             print('error')
#     test(bert_bilstm,(test_texts,test_labels),num_neg,num_pos,epoch)

In [66]:
def predict(epoch,model):
    num_pos,num_neg=0,0
    for i in test_labels:
        if i==0:
            num_neg+=1
        elif i==1:
            num_pos+=1
        else:
            print('error')
    test(model,(test_texts,test_labels),num_neg,num_pos,epoch)

In [69]:
# num_pos,num_neg=0,0
# for i in test_labels:
#     if i==0:
#         num_neg+=1
#     elif i==1:
#         num_pos+=1
#     else:
#         print('error')
# notrain(bert_textcnn,(test_texts,test_labels),num_pos,num_neg)

process:180/180
正类的准确率： 0.5384615384615384
负类的准确率： 0.4473684210526316
整体的准确率： 0.5


In [74]:
model=train(bert_textcnn)

========== epoch:1/5 ==========
process:351/359 | loss=0.38425526693463324 | lr=1e-05
开始预测...
process:180/180
正类的准确率： 0.9326923076923077
负类的准确率： 0.8157894736842105
整体的准确率： 0.8833333333333333
========== epoch:2/5 ==========
process:351/359 | loss=0.1167408867739141 | lr=6e-0666
开始预测...
process:180/180
正类的准确率： 0.9326923076923077
负类的准确率： 0.8947368421052632
整体的准确率： 0.9166666666666666
========== epoch:3/5 ==========
process:351/359 | loss=0.09991640066727996 | lr=3.6e-066999999996e-06
开始预测...
process:180/180
正类的准确率： 0.9615384615384616
负类的准确率： 0.8289473684210527
整体的准确率： 0.9055555555555556
========== epoch:4/5 ==========
process:351/359 | loss=0.012132874061353504 | lr=2.1599999999999996e-06
开始预测...
process:180/180
正类的准确率： 0.875
负类的准确率： 0.868421052631579
整体的准确率： 0.8722222222222222
========== epoch:5/5 ==========
process:351/359 | loss=0.08050419825594872 | lr=1.2959999999999997e-066
开始预测...
process:180/180
正类的准确率： 0.8846153846153846
负类的准确率： 0.8289473684210527
整体的准确率： 0.8611111111111112


| model | finetuning | batch_size | epoch | init_lr | max_length | unit\filter_num | filter_size | dropout | data | acc_pos | acc_neg | acc |
| :----: | :-----: | :----: | :----: | :----: | :-----: | :----: | :----: | :----: | :----: | :----: | :----: | :----: |
| bert | N | - | - | - | - | - | - | - | 重新标注的900条数据 | 0.86 | 0.13 | 0.55 |
| bert | Y | 16 | 5 | 2e-5 | 128 | - | - | 0.3 | - | 0.82 | 0.92 | 0.88 |
| bert | Y | 16 | 5 | 1e-5 | 128 | - | - | 0.3 | - |  |  |  |
| bert | Y | 16 | 5 | 5e-6 | 128 | - | - | 0.3 | - | 0.9 | 0.9 | 0.9 |
| bert | Y | 4 | 5 | 2e-5 | 128 | - | - | 0.3 | 重新标注的900条数据 | 0.88 | 0.93 | 0.905 |
| bert | Y | 4 | 5 | 1e-5 | 128 | - | - | 0.3 | 重新标注的900条数据 | 0.91 | 0.92 | 0.916 |
| bert | Y | 4 | 5 | 5e-6 | 128 | - | - | 0.3 | 重新标注的900条数据 | 0.92 | 0.89 | 0.91 |
| bert | Y | 4 | 5 | 1e-5/1e-4(分层) | 128 | - | - | 0.3 | 重新标注的900条数据 | 0.89 | 0.94 | 0.916 |
| bert-bilstm | N | - | - | - | - | - | - | - | 重新标注的900条数据 | 0.71 | 0.23 | 0.51 |
| bert-bilstm | Y | 16 | 5 | 2e-5 | 128 | 128 | - | 0.2 | - | 0.76 | 0.94 | 0.87 |
| bert-bilstm | Y | 16 | 5 | 2e-5 | 256 | 128 | - | 0.2 | - | 0.83 | 0.9 | 0.87 |
| bert-bilstm | Y | 16 | 5 | 2e-5 | 256 | 128 | - | 0.2 | 数据清洗 | 0.81 | 0.87 | 0.84 |
| bert-bilstm | Y | 16 | 5 | 2e-5 | 128 | 128 | - | 0.3 | 数据清洗 | 0.8 | 0.88 | 0.85 |
| bert-bilstm | Y | 16 | 5 | 1.2e-5 | 128 | 128 | - | 0.3 | 数据清洗 | 0.88 | 0.9 | 0.9 |
| bert-bilstm | Y | 16 | 5 | 1e-5 | 150 | 128 | - | 0.3 | 数据清洗 | 0.89 | 0.9 | 0.9 |
| bert-bilstm | Y | 16 | 5 | 5e-6 | 150 | 128 | - | 0.3 | 数据清洗 | 0.87 | 0.92 | 0.9 |
| bert-bilstm | Y | 4 | 5 | 5e-6 | 150 | 128 | - | 0.3 | 重新标注的900条数据 | 0.93 | 0.86 | 0.9 |
| bert-bilstm | Y | 2 | 5 | 1e-5 | 150 | 128 | - | 0.3 | 重新标注的900条数据 | 0.93 | 0.89 | 0.916 |
| bert-bilstm | Y | 4 | 5 | 1e-5 | 150 | 128 | - | 0.3 | 重新标注的900条数据 | 0.91 | **0.96** | **0.933** |
| bert-bilstm | Y | 8 | 5 | 1e-5 | 150 | 128 | - | 0.3 | 重新标注的900条数据 | 0.93 | 0.89 | 0.916 |
| bert-bilstm | Y | 4 | 5 | 2e-5 | 150 | 128 | - | 0.3 | 重新标注的900条数据 | 0.9 | 0.88 | 0.89 |
| bert-bilstm | Y | 4 | 5 | 1e-5/1e-4(分层) | 150 | 128 | - | 0.3 | 重新标注的900条数据 | 0.92 | 0.93 | 0.927 |
| bert-textcnn | N | - | - | - | - | - | - | - | 重新标注的900条数据 | 0.53 | 0.44 | 0.5 |
| bert-textcnn | Y | 2 | 5 | 1e-5 | 150 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.93 | 0.89 | 0.916 |
| bert-textcnn | Y | 4 | 5 | 1e-5 | 150 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.93 | 0.92 | 0.927 |
| bert-textcnn | Y | 8 | 5 | 1e-5 | 150 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.93 | 0.89 | 0.916 |
| bert-textcnn | Y | 4 | 5 | 1e-5 | 150 | 128 | [4,5,6] | 0.3 | 重新标注的900条数据 | 0.92 | 0.92 | 0.92 |
| bert-textcnn | Y | 4 | 5 | 1e-5 | 150 | 100 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.9 | 0.93 | 0.916 |
| bert-textcnn | Y | 4 | 5 | 1e-5 | 128 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.92 | 0.92 | 0.92 |
| bert-textcnn | Y | 4 | 5 | 2e-5 | 150 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | **0.96** | 0.84 | 0.91 |
| bert-textcnn | Y | 4 | 5 | 2e-5 | 128 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.903 | 0.907 | 0.905 |
| bert-textcnn | Y | 2 | 5 | 1e-5/1e-4(分层) | 128 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.89 | 0.89 | 0.89 |
| bert-textcnn | Y | 4 | 5 | 1e-5/1e-4(分层) | 128 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | **0.96** | 0.89 | **0.933** |
| bert-textcnn | Y | 8 | 5 | 1e-5/1e-4(分层) | 128 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.93 | 0.92 | 0.927 |
| bert-textcnn | Y | 4 | 5 | 1e-5/1e-4(分层) | 150 | 128 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.93 | 0.9 | 0.92 |
| bert-textcnn | Y | 4 | 5 | 1e-5/1e-4(分层) | 150 | 128 | [4,5,6] | 0.3 | 重新标注的900条数据 | 0.92 | 0.85 | 0.89 |
| bert-textcnn | Y | 4 | 5 | 1e-5/1e-4(分层) | 150 | 100 | [3,4,5] | 0.3 | 重新标注的900条数据 | 0.95 | 0.828 | 0.9 |

bert

- batch_size=16
- epoch=5
- lr=2e-5
- max_length=128
- dropout=0.3
- 结果:
  - 正类的准确率： 0.8203125
  - 负类的准确率： 0.9186046511627907
  - 整体的准确率： 0.8809808612440191
  
- batch_size=16
- epoch=5
- lr=5e-6
- max_length=128
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9038461538461539
  - 负类的准确率： 0.9078947368421053
  - 整体的准确率： 0.9055555555555556
  
- batch_size=4
- epoch=5
- lr=2e-5
- max_length=128
- dropout=0.3
- 数据：900条
- 结果：
  - 正类的准确率： 0.8846153846153846
  - 负类的准确率： 0.9342105263157895
  - 整体的准确率： 0.9055555555555556
  
- batch_size=4
- epoch=5
- lr=1e-5
- max_length=128
- dropout=0.3
- 数据：900条
- 结果：
  - 正类的准确率： 0.9134615384615384
  - 负类的准确率： 0.9210526315789473
  - 整体的准确率： 0.9166666666666666
  
- batch_size=4
- epoch=5
- lr=5e-6
- max_length=128
- dropout=0.3
- 数据：900条
- 结果：
  - 正类的准确率： 0.9230769230769231
  - 负类的准确率： 0.8947368421052632
  - 整体的准确率： 0.9111111111111111
  
- batch_size=4
- epoch=5
- lr=1e-5/1e-4
- max_length=128
- dropout=0.3
- 数据：900条
- 结果：
  - 正类的准确率： 0.8942307692307693
  - 负类的准确率： 0.9473684210526315
  - 整体的准确率： 0.9166666666666666
---

bert-bilstm

- 无训练数据
- 注：使用同样的900条数据，对0.2的数据进行预测
- 结果
  - 正类的准确率： 0.7115384615384616
  - 负类的准确率： 0.23684210526315788
  - 整体的准确率： 0.5111111111111111

- batch_size=16
- epoch=5
- lr=2e-5
- max_length=128
- unit=128
- dropout=0.2
- 结果：
  - 正类的准确率： 0.765625
  - 负类的准确率： 0.9457364341085271
  - 整体的准确率： 0.8767942583732058
  

- batch_size=16
- epoch=5
- lr=2e-5
- max_length=256
- unit=128
- dropout=0.2
- 结果：
  - 正类的准确率： 0.828125
  - 负类的准确率： 0.8992248062015504
  - 整体的准确率： 0.8720095693779905
  
- batch_size=16
- epoch=5
- lr=2e-5
- max_length=256
- unit=128
- dropout=0.2
- 去除@#url重复
- 结果：
  - 正类的准确率： 0.8125
  - 负类的准确率： 0.8682170542635659
  - 整体的准确率： 0.84688995215311
  
- batch_size=16
- epoch=5
- lr=2e-5
- max_length=128
- unit=128
- dropout=0.3
- 去除@#url重复html
- 结果：
  - 正类的准确率： 0.8046875
  - 负类的准确率： 0.8827519379844961
  - 整体的准确率： 0.8528708133971292
 
- batch_size=16
- epoch=5
- lr=1.2e-5
- max_length=128
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 结果：
  - 正类的准确率： 0.8817034700315457
  - 负类的准确率： 0.9104186952288218
  - 整体的准确率： 0.8994581577363034
  
- batch_size=16
- epoch=5
- lr=1e-5
- max_length=150
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 注：好像没有重新创建模型
- 结果：
  - 正类的准确率： 0.8974763406940063
  - 负类的准确率： 0.9065238558909445
  - 整体的准确率： 0.9030704394942806

- batch_size=16
- epoch=5
- lr=5e-6
- max_length=150
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 注：好像没有重新创建模型，只有一个的结果比较高
- 结果：
  - 正类的准确率： 0.8769716088328076
  - 负类的准确率： 0.9211295034079844
  - 整体的准确率： 0.9042745334136063
  
- batch_size=4
- epoch=5
- lr=5e-6
- max_length=150
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 注：使用的是新标的900条数据
- 结果：
  - 正类的准确率： 0.9326923076923077
  - 负类的准确率： 0.868421052631579
  - 整体的准确率： 0.9055555555555556
  
- batch_size=4
- epoch=5
- lr=1e-5
- max_length=150
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 注：使用的是新标的900条数据
- 结果：
  - 正类的准确率： 0.9134615384615384
  - 负类的准确率： 0.9605263157894737
  - 整体的准确率： 0.9333333333333333

- batch_size=4
- epoch=5
- lr=2e-5
- max_length=150
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 注：使用的是新标的900条数据
- 结果：
  - 正类的准确率： 0.9038461538461539
  - 负类的准确率： 0.881578947368421
  - 整体的准确率： 0.8944444444444445

- batch_size=4
- epoch=5
- lr=1e-5/1e-4
- max_length=150
- unit=128
- dropout=0.3
- 去除@#url重复html空白行
- 注：使用的是新标的900条数据
- 结果：
  - 正类的准确率： 0.9230769230769231
  - 负类的准确率： 0.9342105263157895
  - 整体的准确率： 0.9277777777777778
---
bert_textcnn

- batch_size=4
- epoch=5
- lr=1e-5
- max_length=150
- filter_sizes=[3,4,5]
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9326923076923077
  - 负类的准确率： 0.9210526315789473
  - 整体的准确率： 0.9277777777777778
  
- batch_size=4
- epoch=5
- lr=2e-5
- max_length=150
- filter_sizes=[3,4,5]
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9615384615384616
  - 负类的准确率： 0.8421052631578947
  - 整体的准确率： 0.9111111111111111
  
- batch_size=4
- epoch=5
- lr=5e-6
- max_length=150
- filter_sizes=[3,4,5]
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9230769230769231
  - 负类的准确率： 0.868421052631579
  - 整体的准确率： 0.9
  
- batch_size=4
- epoch=5
- 分层学习率：lr=1e-5/1e-4
- max_length=150
- filter_sizes=[3,4,5]
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9326923076923077
  - 负类的准确率： 0.9078947368421053
  - 整体的准确率： 0.9222222222222223
  
- batch_size=4
- epoch=5
- lr=1e-5
- max_length=150
- filter_sizes=[4,5,6]
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9230769230769231
  - 负类的准确率： 0.9210526315789473
  - 整体的准确率： 0.9222222222222223
  
- batch_size=4
- epoch=5
- lr=1e-5
- max_length=150
- filter_sizes=[4,5,6]
- dropout=0.3
- num_filter=100
- 结果：
  - 正类的准确率： 0.9038461538461539
  - 负类的准确率： 0.9342105263157895
  - 整体的准确率： 0.9166666666666666
  
- batch_size=4
- epoch=5
- 分层学习率：lr=1e-5/1e-4
- max_length=150
- filter_sizes=[4,5,6]
- dropout=0.3
- 结果：
  - 正类的准确率： 0.9230769230769231
  - 负类的准确率： 0.8552631578947368
  - 整体的准确率： 0.8944444444444445

- batch_size=4
- epoch=5
- 分层学习率：lr=1e-5/1e-4
- max_length=150
- filter_sizes=[3,4,5]
- dropout=0.3
- num_filter=100
- 结果：
  - 正类的准确率： 0.9519230769230769
  - 负类的准确率： 0.8289473684210527
  - 整体的准确率： 0.9
  
- batch_size=4
- epoch=5
- lr=1e-5
- max_length=128
- filter_sizes=[3,4,5]
- dropout=0.3
- num_filter=100
- 结果：
  - 正类的准确率： 0.9230769230769231
  - 负类的准确率： 0.9210526315789473
  - 整体的准确率： 0.9222222222222223


- batch_size=4
- epoch=5
- lr=2e-5
- max_length=128
- filter_sizes=[3,4,5]
- dropout=0.3
- num_filter=100
- 结果：
  - 正类的准确率： 0.9038461538461539
  - 负类的准确率： 0.9078947368421053
  - 整体的准确率： 0.9055555555555556


- batch_size=4
- epoch=5
- 分层学习率：lr=1e-5/1e-4
- max_length=128
- filter_sizes=[3,4,5]
- dropout=0.3
- num_filter=100
- 结果：
  - 正类的准确率： 0.9615384615384616
  - 负类的准确率： 0.8947368421052632
  - 整体的准确率： 0.9333333333333333